In [1]:
import networkx as nx
from operator import add, sub
import numpy as np
import matplotlib.pyplot as plt
from python.SpringRank import SpringRank
import python.tools as tl

def get_space_ranks(graph, self_edge_weight = 1):
    #intialise dictionary
    node_page = {}
    for node in list(graph.nodes()):
        node_page[node] = [0.0] * graph.number_of_nodes()

    #calculating what each nodes page will look like
    i = 0
    while True:
        i = i+1
        change = 0
        for node in list(graph.nodes()):
            norm_const = 1/(graph.in_degree(node, weight='weight')+self_edge_weight)
            new_page_draft = [0.0] * graph.number_of_nodes()
            new_page_draft[node] = norm_const
            for superior in list(graph.predecessors(node)):
                edge_weight = graph.get_edge_data(superior, node)["weight"]
                new_page_draft = list(map(add, new_page_draft, [edge_weight*(norm_const)*x for x in node_page[superior]]))

            change = change + sum(list(map(sub, node_page[node], new_page_draft)))
            node_page[node] = new_page_draft

        if i%100 == 0:
            print("Currently at iteration ", i)

        if abs(change) < 1:
            print("stopped at iteration:", i)
            break

    #now calculating each nodes "page" rank values
    #output is list with node i's value at position i
    ranking = [0.0] * graph.number_of_nodes()
    for node in node_page.keys():
        ranking = list(map(add, ranking, node_page[node]))

    return ranking

def get_spring_ranks(graph, alpha=0., l0=1., l1=1.):
    nodes = list(graph.nodes())
    A = nx.to_scipy_sparse_matrix(graph, dtype=float,nodelist=nodes)
    ranking =SpringRank(A,alpha=alpha,l0=l0,l1=l1)
    ranking =tl.shift_rank(ranking)